##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Сохранение и восстановление моделей

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотри на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запусти в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучай код GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/r2/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачай ноутбук</a>
  </td>
</table>

Прогресс обучения модели может быть сохранен во время и после обучения. Это значит, что можно продложить обучение с того места, где оно было остановлено, что позволит избежать длительных безостановочных сессий. Сохранение также значит то, что вы можете поделиться своей моделью с другими и они смогут воспроизвести вашу работу. При публикации исследовательских моделей и техник большинство практиков машинного обучения делятся:

* кодом необходимым для создания модели, и
* обученными весами, или параметрами модели

Публикация этих данных помогает другим понять как работает модель и попробовать ее самостоятельно с новыми данными.

Предупреждение: Будьте осторожны с ненадежным кодом — модели TensorFlow являются кодом. См. [Безопасное использование TensorFlow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) для подробностей.

### Варианты

Существуют различные способы сохранения моделей Tensorflow - зависит от API которое ты используешь. Это руководство использует [tf.keras](https://www.tensorflow.org/guide/keras) высокоуровневый API для построения и обучения моделей в TensorFlow. Для остальных подходов см. руководство TensorFlow [сохранение и восстановление](https://www.tensorflow.org/guide/saved_model) или [Сохранение в eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Установка

### Инсталляция и импорт

Установим и импортируем TensorFlow и зависимые библиотеки:

In [0]:
try:
  # %tensorflow_version существует только в Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Требуется для сохранения модели в формате HDF5

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Получите набор данных

Для демонстрации того как сохранять и загружать веса вы используете [датасет MNIST](http://yann.lecun.com/exdb/mnist/). Для ускорения запусков используйте только первые 1000 примеров:

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Определите модель

Начните с построения простой последовательной модели:

In [0]:
# Определим простую последовательную модель
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Создадим экземпляр базовой модели
model = create_model()

# Распечатаем архитектуру модели
model.summary()

## Сохраните чекпоинты во время обучения

Вы можете использовать обученную модель без повторного ее обучения, или продолжить обучение с того места где вы остановились, в случае если процесс обучения был прерван. Коллбек `tf.keras.callbacks.ModelCheckpoint` позволяет непрерывно сохранять модель как *во время* так и *по окончанию* обучения

### Использование чекпоинт коллбека (checkpoint callback)

Создайте коллбек `tf.keras.callbacks.ModelCheckpoint` который сохраняет веса только во время обучения:

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Создаем коллбек сохраняющий веса модели
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Обучаем модель с новым коллбеком
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# Это может генерировать предупреждения относящиеся к сохранению состояния оптимизатора.
# Эти предупреждения (и подобные предупреждения в этом уроке)
# используются для предотвращения устаревшего использования и могут быть проигнорированы.

Это создаст единый набор чекпоинт файлов TensorFlow который обновляется в конце каждой эпохи:

In [0]:
!ls {checkpoint_dir}

Создайте новую, необученную модель. При восстановлении модели только из весов, у вас должна быть модель с такой же архитектурой как и восстанавливаемая. Так как архитектура модели такая же вы можете поделиться весами несмотря на то что это другой *экземпляр* модели.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [0]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Then load the weights from the checkpoint and re-evaluate:

In [0]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [0]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

Now, look at the resulting checkpoints and choose the latest one:

In [0]:
! ls {checkpoint_dir}

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [0]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoints](../../guide/keras/checkpoints) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/saving_and_serializing#weights-only_saving_in_savedmodel_format) guide):

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Save the entire model

The model and optimizer can be saved to a file that contains both their state (weights and variables) and the model configuration. This allows you to export a model so it can be used without access to the original Python code. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5](https://js.tensorflow.org/tutorials/import-keras.html), [Saved Model](https://js.tensorflow.org/tutorials/import-saved-model.html)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

### Save model as an HDF5 file

Keras also provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob:

In [0]:
# Create a new model instance
model = create_model()

# Train the model
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
model.save('my_model.h5')

Now, recreate the model from that file:

In [0]:
# Recreate the exact same model, including its weights and the optimizer
new_model = keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Check its accuracy:

In [0]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.


### As a `saved_model`

Caution: This method of saving a `tf.keras` model is experimental and may change in future versions.

Build a new model, then train it:

In [0]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Create a `saved_model`, and place it in a time-stamped directory with `tf.keras.experimental.export_saved_model`:

In [0]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

List your saved models:

In [0]:
!ls saved_models/

Reload a fresh Keras model from the saved model:

In [0]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)

# Check its architecture
new_model.summary()

Run a prediction with the restored model:

In [0]:
model.predict(test_images).shape

In [0]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # Keep the optimizer that was loaded
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))